# Read Kommersant

https://www.kommersant.ru/ratings

In [1]:
%run common.py

import matplotlib.pyplot as plt
import locale
#locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

print(locale.getlocale())

eng_co=sa.create_engine('sqlite+pysqlite:///eco_words.sqlite3')#, echo=True)
stop_w.update(['т.', 'д.', 'д..', 'ъ', 'это', 'тыс', 'руб', 'году', 'год', 'года', 'который', 
               'которому', 'которая', 'которым', 'которыми', 'считает', 'говорит'])

def update_stop_words():
    lst_txt_files=glob.glob('4text/*.txt')
    stopd_words_ru=set()

    for f in lst_txt_files:
        with open(f, 'r') as sw_file:
            stopd_words_ru.update(sw_file.read().split('\n'))


    stop_words2={w for w in stopd_words_ru if w.find(' ') > 0}
    return {w for w in stopd_words_ru if w not in stop_words2}, stop_words2

stop_w1, stop_w2=update_stop_words()

remove_stop_words2=re.compile(r'(?i)' + '|'.join({ r'(\b{}\b)'.format(w) for w in stop_w2}))
stop_w.update(stop_w1)
#print(stop_w)

%matplotlib notebook

('ru_RU', 'UTF-8')


In [78]:
strKomm=r'https://www.kommersant.ru/ratings'

def get_kommersant_ribbon(strURL=strKomm):
    
    def get_item(strUrl):
        r=sessKomm.get(strUrl)
        soup=BS(r.text, 'html.parser')
        
        title=soup.find('meta', {'name':'title'})['content']

        re_cut=re.compile('(?m)(?<={title}).*$'.format(title=title.replace('.', '\.')))
        descr=soup.find('meta', {'name':'description'})['content']

        try:
            scut=re_cut.search(descr)[0].strip()
        except:
            pass
        print(title)
        print(descr)

        key_w=soup.find('meta', {'name':'keywords'})['content']
    
        div_text=soup.find('div', class_='text')
        div_item=soup.find('div', class_='article_text_wrapper')
    
        try:
            short=soup.find('span', class_='b-article__intro').text.strip()
        except:
            short=None
        
        heads='\n'.join([h.text.strip() for h in soup.findAll('h1', class_="article_subheader")]) + \
        '\n'+ '\n'.join([th.text.strip() for th in soup.findAll('div', class_='b-article__thought')])
    
        ptext=soup.findAll('p', class_='b-article__text')
        text='\n'.join(p.text.strip() for p in ptext if not p.find('span', class_='b-article__intro'))
    
        return {'text':text, 'heads':heads, 'cuts':scut, 'keyw':key_w, 'short':short}
    
    sessKomm=requests.session()
    sessKomm.headers.update(req_headers)
    
    r=sessKomm.get(strURL)
    soup=BS(r.text, 'html.parser')

    items=soup.findAll('div', class_='ratings_docs__item')
    ret_lst=[]
    for item in items:
        
        str_dt= re.match(r'.*, (?P<day>\d{1,2})\.(?P<month>\d{1,2})\.(?P<year>\d{4})', 
                    item.find('span', class_='ratings_docs__status_name').text.strip())
        
        #print(str_dt, str_dt['year'], str_dt['month'], str_dt['day'])
        
        dt=date(int(str_dt['year']), int(str_dt['month']), int(str_dt['day']))
        title=item.find('div', class_='ratings_docs__item_header').text
        #print(title)
        try:
            subtitle=item.find('div', class_='ratings_docs__item_subheader').text
        except:
            subtitle=None
        view_rating=item.find('span', class_='ratings_hitscount_number').text
        
        #short=item.find('div', class_='ratings_docs__item_vvodka').text
        link=r'https://www.kommersant.ru/' + item.find('a', class_='ratings_docs__text')['href']
        dct={'title':title, 'rubrica': subtitle, 'link':link, 
               'date':dt, 'rating':view_rating}
        dct.update(get_item(link))
        ret_lst.append(dct)
        
    time.sleep(2)
    return ret_lst
    
    
kommersant=ArticlesDataFrame(pd.DataFrame(get_kommersant_ribbon()).set_index('title'))
kommersant.to_sql('KOMMERSANT', con=eng_co, if_exists='upsert')
print(kommersant.head(5))

Доллар скинул «санкционные» рубли - Но укрепление российской валюты в дальнейшем проблематично - Газета "Коммерсантъ" - Издательский Дом КоммерсантЪ.
Доллар скинул «санкционные» рубли - Но укрепление российской валюты в дальнейшем проблематично - Газета "Коммерсантъ" - Издательский Дом КоммерсантЪ. Неделя завершилась уверенным укреплением российской валюты. В пятницу курс доллара на Московской бирже закрылся возле отметки 65,85 руб./$ — минимального значения с начала ноября. С начала недели американская валюта потеряла более 2 руб. Перенос очередной порции антироссийских санкций на следующий год спровоцировал инвесторов на сокращение валютных позиций. В ближайшее время курс доллара может опуститься ниже уровня 65 руб./$, но тема санкций сохранит давление на российскую валюту.
Владимир Путин прозвенел на весь мир - До каких рамок дошли переговоры на саммите Россия—АСЕАН - Газета "Коммерсантъ" - Издательский Дом КоммерсантЪ.
Владимир Путин прозвенел на весь мир - До каких рамок дошли пер

UnboundLocalError: local variable 'scut' referenced before assignment

In [81]:
strT='РАНХиГС: почти четверть россиян живет в режиме жесткой экономии - ' \
      'Лента новостей (Москва) - Издательский Дом КоммерсантЪ.'
strF='РАНХиГС: почти четверть россиян живет в режиме жесткой экономии - ' \
'Лента новостей (Москва) - Издательский Дом КоммерсантЪ. ' \
'Эксперты РАНХиГС провели анализ социально-экономического положения и самочувствия россиян и пришли к выводу, ' \
'что больше 70% из них вынуждены экономить. Уровень доходов 22% участников проведенного исследователями опроса ' \
'не позволяет им приобрести даже необходимый базовый набор продуктов питания'


fnd=re.compile('(?m:)(?<={title}).*$'.format(title=strT.replace('.', '\.')))

print(fnd.search(strF.replace('<BR>', '\n')))


РАНХиГС: почти четверть россиян живет в режиме жесткой экономии - Лента новостей (Москва) - Издательский Дом КоммерсантЪ. Эксперты РАНХиГС провели анализ социально-экономического положения и самочувствия россиян и пришли к выводу, что больше 70% из них вынуждены экономить. Уровень доходов 22% участников проведенного исследователями опроса не позволяет им приобрести даже необходимый базовый набор продуктов питания
None


### Kommersant from DB

In [ ]:
kommersant_db=pd.read_sql('KOMMERSANT', con=eng_co, index_col='title')
kommersant_db['date']=pd.to_datetime(kommersant_db['date'], format='%Y-%m-%d')
#print(kommersant_db)
kommersant_db=kommersant_db[(kommersant_db['date'].isin(dateFilter)) & (kommersant_db['rating'].astype(int)>1000)]
print(kommersant_db)

In [ ]:
def make_komm_text(pdf=kommersant_db):
    sessExp=requests.session()
    sessExp.headers.update(req_headers)
    text=''
    
    for i, r in pdf.iterrows():
        time.sleep(1)
        r=sessExp.get(r['link'])
        soup=BS(r.text, 'html.parser')
        ps=soup.findAll('p', class_='b-article__text')
        divs=soup.findAll('div', class_='b-article__thought')
        text+='  '.join(p.text for p in ps if p) + '  '
        text+='  '.join(d.text for d in divs if d) + '  ' 
    return text
komm_text=make_komm_text()


In [ ]:
def FreqWordCollacs(text, win_size=2, stop_words=stop_w, nbest_ration=50):
    
    x_text=re.sub(r'\W', ' ', text)
    
    word_filter=lambda x: x.lower() in stop_words or x.find('.')>=0

    w_token=[w.replace('.-', '') for w in word_tokenize(x_text) if w.lower() not in stop_words]
    
    bcf=BigramCollocationFinder.from_words(w_token, window_size=win_size)
    bcf.apply_word_filter(word_filter)
    bigrams=bcf.nbest(BigramAssocMeasures.likelihood_ratio, nbest_ration)
    
    tokenizer=MWETokenizer(bigrams)

    #print(tokenizer.tokenize(w_token))
    
    freq=nltk.FreqDist(tokenizer.tokenize(w_token))

    freq={i[0].replace('_', ' '):i[1] for i in freq.items()}
    
    for i in bigrams:
        base_key=' '.join(i)
        keys=list(freq.keys())
        for j in keys:
            lst=j.split(' ')
            if len(lst)>1:
                try:
                    dist=nltk.jaccard_distance(set(i[0]), set(lst[0])) + nltk.jaccard_distance(set(i[1]), set(lst[1]))
                    if 0 < dist < 0.4:
                        freq[base_key]+=freq.pop(j, 0)
                        #print(base_key, j, freq[base_key], dist)
                except KeyError:
                    pass
                    #print('error: ', base_key, j)
    return nltk.FreqDist(freq)

item_kc=remove_stop_words2.sub(' ', komm_text)

#stop_w.update(['это', '—', "''", '»', '``', '«', 'года', '“', 'ъ', '”', '..', 'также', 'могут', 
#               'мнению', 'говорит', 'делать', 'время', 'которые', 'числе', 'октября', 'лет', 
#               'пояснил', 'словам', 'тыс', 'тыс.', 'руб', 'руб.', 'просто', 'например', 'скорее', 
#               'около', 'будут', 'говорили', 'которого', 'господин', 'сразу', 'которая', 
#               'которое', 'которому', 'которые', 'который', 'редко', 'крепко', 'далее','одной', 
#               'одна','одним', 'одни', 'стали','стала', 'стал', 'стало', 'выше','свыше', 'самой', 
#               'самим', 'саму','самый','самого', 'кроме', 'проще', 'вроде', 
#               'затем','исходя','частое','касается', 'вслед','неудобно','удобно','некоторым',
#               'некоторой','некоторое','сюда','туда','оттуда','отсюда','своему','своим','своего','свою'] )


kommers_freq=FreqWordCollacs(item_kc, stop_words=stop_w)

show_cloud(freq_array=kommers_freq, 
           title='Темы месяца - Коммерсант', max_words=200, fig_size=(9, 5), filename='kommers.png')

In [ ]:
item_exp=remove_stop_words2.sub(' ', expert_text)
#item_kc=remove_stop_words2.sub(' ', komm_text)

expert_freq=FreqWordCollacs(item_exp, stop_words=stop_w) 
#kommers_freq=FreqWordCollacs(item_kc, stop_words=stop_w)


#keys_komm=set(kommers_freq.keys())
#keys_exp=set(expert_freq)
#for k in (keys_komm & keys_exp):
#    kommers_freq[k]+=expert_freq.pop(k, 0)

#print(keys_komm & keys_exp)
#kommers_freq.update(expert_freq)

show_cloud(freq_array=expert_freq, 
           title='Темы месяца - Два источника', max_words=200, fig_size=(9, 5), filename='kommers.png')

In [4]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/egor/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.


True

In [ ]:
#https://wiki-ru-wordnet.readthedocs.io/en/latest/
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()


In [82]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
